### OCI Data Science - Useful Tips
<details>
<summary><font size="2">Check for Public Internet Access</font></summary>

```python
import requests
response = requests.get("https://oracle.com")
assert response.status_code==200, "Internet connection failed"
```
</details>
<details>
<summary><font size="2">Helpful Documentation </font></summary>
<ul><li><a href="https://docs.cloud.oracle.com/en-us/iaas/data-science/using/data-science.htm">Data Science Service Documentation</a></li>
<li><a href="https://docs.cloud.oracle.com/iaas/tools/ads-sdk/latest/index.html">ADS documentation</a></li>
</ul>
</details>
<details>
<summary><font size="2">Typical Cell Imports and Settings for ADS</font></summary>

```python
%load_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import logging
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.ERROR)

import ads
from ads.dataset.factory import DatasetFactory
from ads.automl.provider import OracleAutoMLProvider
from ads.automl.driver import AutoML
from ads.evaluations.evaluator import ADSEvaluator
from ads.common.data import ADSData
from ads.explanations.explainer import ADSExplainer
from ads.explanations.mlx_global_explainer import MLXGlobalExplainer
from ads.explanations.mlx_local_explainer import MLXLocalExplainer
from ads.catalog.model import ModelCatalog
from ads.common.model_artifact import ModelArtifact
```
</details>
<details>
<summary><font size="2">Useful Environment Variables</font></summary>

```python
import os
print(os.environ["NB_SESSION_COMPARTMENT_OCID"])
print(os.environ["PROJECT_OCID"])
print(os.environ["USER_OCID"])
print(os.environ["TENANCY_OCID"])
print(os.environ["NB_REGION"])
```
</details>

In [ ]:
import requests
from bs4 import BeautifulSoup
import json
import os
import ipywidgets as widgets
from IPython.display import display, clear_output

# Replace with your actual Google API key and Search Engine ID
GOOGLE_API_KEY = 'YOUR_GOOGLE_API_KEY'
SEARCH_ENGINE_ID = 'YOUR_SEARCH_ENGINE_ID'
GOOGLE_SEARCH_URL = "https://www.googleapis.com/customsearch/v1"

# Bloom's taxonomy verbs
BLOOM_VERBS = {
    "Remember": ["list", "define", "describe", "state", "identify", "label", "name"],
    "Understand": ["summarize", "explain", "classify", "compare", "contrast", "discuss"],
    "Apply": ["use", "solve", "demonstrate", "apply", "show", "complete"],
    "Analyze": ["analyze", "compare", "contrast", "examine", "test", "experiment"],
    "Evaluate": ["judge", "criticize", "defend", "justify", "assess", "evaluate"],
    "Create": ["design", "construct", "create", "develop", "formulate", "plan"]
}

class EducationalTopic:
    def __init__(self, title, introduction, questions):
        self.title = title
        self.introduction = introduction
        self.questions = questions

class TopicManager:
    def __init__(self):
        self.ensure_fallback_file_exists()

    def ensure_fallback_file_exists(self):
        if not os.path.exists('fallback_topics.json'):
            self.create_fallback_file()

    def create_fallback_file(self):
        fallback_data = [
            {
                "title": "Sample Topic 1",
                "introduction": "Introduction text for Sample Topic 1",
                "questions": {
                    "Remember": ["Question 1", "Question 2"],
                    "Understand": ["Question 1", "Question 2"],
                    "Apply": ["Question 1", "Question 2"],
                    "Analyze": ["Question 1", "Question 2"],
                    "Evaluate": ["Question 1", "Question 2"],
                    "Create": ["Question 1", "Question 2"]
                }
            },
            {
                "title": "Sample Topic 2",
                "introduction": "Introduction text for Sample Topic 2",
                "questions": {
                    "Remember": ["Question 1", "Question 2"],
                    "Understand": ["Question 1", "Question 2"],
                    "Apply": ["Question 1", "Question 2"],
                    "Analyze": ["Question 1", "Question 2"],
                    "Evaluate": ["Question 1", "Question 2"],
                    "Create": ["Question 1", "Question 2"]
                }
            },
            {
                "title": "Web Scraping",
                "introduction": "Web scraping is the process of extracting data from websites. This can be done manually by a user or automatically by a bot. Web scraping is useful for various applications such as data mining, market research, and content aggregation.",
                "questions": {
                    "Remember": ["What is web scraping?", "Name a popular library used for web scraping in Python."],
                    "Understand": ["Explain why web scraping can be useful.", "Describe the ethical considerations one must take into account when web scraping."],
                    "Apply": [
                        "Write a basic Python script using BeautifulSoup to extract data from a website.",
                        "Demonstrate how to handle HTTP errors when web scraping."
                    ],
                    "Analyze": ["Compare web scraping with API usage for data extraction.", "Analyze the potential risks of web scraping to the website being scraped."],
                    "Evaluate": ["Evaluate the impact of web scraping on website performance.", "Assess the legal implications of web scraping."],
                    "Create": ["Create a web scraper that collects data from multiple pages of a website.", "Design a strategy to avoid getting blocked by websites while scraping."]
                }
            }
        ]
        with open('fallback_topics.json', 'w') as file:
            json.dump(fallback_data, file, indent=4)
        print("Created fallback_topics.json with sample data.")

    def fetch_topic(self, topic_name):
        try:
            search_results = self.search_google(topic_name)
            if 'items' in search_results:
                first_result_url = search_results['items'][0]['link']
                return self.scrape_url(first_result_url)
            else:
                return []
        except ValueError as ve:
            print(ve)
            return []
        except requests.exceptions.HTTPError as e:
            print(f"An HTTP error occurred while fetching the topic: {e}")
            return []
        except Exception as e:
            print(f"An error occurred while fetching the topic: {e}")
            return []

    def search_google(self, query):
        params = {
            'key': GOOGLE_API_KEY,
            'cx': SEARCH_ENGINE_ID,
            'q': query
        }
        response = requests.get(GOOGLE_SEARCH_URL, params=params)
        if response.status_code == 403:
            raise ValueError("Invalid API key or Search Engine ID. Please check your Google API credentials.")
        response.raise_for_status()
        return response.json()

    def scrape_url(self, url):
        response = requests.get(url)
        response.raise_for_status()
        return self.parse_topic_page(response.text)

    def parse_topic_page(self, html):
        soup = BeautifulSoup(html, 'html.parser')

        title_elem = soup.find('title')
        if not title_elem:
            return []

        title = title_elem.text.strip()
        introduction_elem = soup.find('meta', {'name': 'description'})
        introduction = introduction_elem['content'].strip() if introduction_elem else "No introduction available."

        # Extracting questions based on Bloom's taxonomy verbs
        text = soup.get_text().lower()
        questions = {level: [] for level in BLOOM_VERBS}

        for level, verbs in BLOOM_VERBS.items():
            for verb in verbs:
                if verb in text:
                    questions[level].append(f"Identify the context where '{verb}' is used in the text.")

        return [EducationalTopic(title, introduction, questions)]

    def fetch_topics_from_file(self):
        try:
            with open('fallback_topics.json', 'r') as file:
                data = json.load(file)
            return [EducationalTopic(**topic) for topic in data]
        except Exception as e:
            print(f"Error loading fallback topics: {e}")
            return []

class AssessmentManager:
    def __init__(self):
        self.correct_answers = 0
        self.total_questions = 0

    def display_topic(self, topic):
        display(widgets.HTML(f"<h2>Topic: {topic.title}</h2>"))
        display(widgets.HTML(f"<p>{topic.introduction}</p>"))

    def conduct_assessment(self, topic):
        self.correct_answers = 0
        self.total_questions = 0

        for level, questions in topic.questions.items():
            display(widgets.HTML(f"<h3>{level} Level Questions:</h3>"))
            for question in questions:
                display(widgets.HTML(f"<p>{question}</p>"))
                answer = widgets.Text(
                    value='',
                    placeholder='Type your answer',
                    description='Answer:',
                    disabled=False
                )
                display(answer)
                display_button = widgets.Button(description="Submit")
                display(display_button)

                def on_submit(b, ans=answer):
                    if ans.value:
                        self.correct_answers += 1
                    self.total_questions += 1
                    ans.disabled = True
                    display_button.disabled = True

                display_button.on_click(on_submit)
        
        display(widgets.HTML(f"<p>You answered {self.correct_answers} out of {self.total_questions} questions correctly.</p>"))

class MainApp:
    def __init__(self):
        self.topic_manager = TopicManager()
        self.assessment_manager = AssessmentManager()
        self.setup_ui()

    def setup_ui(self):
        self.search_box = widgets.Text(
            value='',
            placeholder='Type something',
            description='Search:',
            disabled=False
        )

        self.search_button = widgets.Button(
            description='Search',
            disabled=False,
            button_style='', # 'success', 'info', 'warning', 'danger' or ''
            tooltip='Search'
        )
        self.search_button.on_click(self.on_search_click)

        display(self.search_box, self.search_button)

    def on_search_click(self, b):
        clear_output()
        topic_name = self.search_box.value
        topics = self.topic_manager.fetch_topic(topic_name)
        if not topics:
            print(f"No topics fetched for '{topic_name}'.")
            fallback_choice = widgets.ToggleButtons(
                options=['Yes', 'No'],
                description='Load fallback topics?',
                disabled=False,
                button_style=''
            )
            fallback_choice.observe(self.on_fallback_choice_change, names='value')
            display(fallback_choice)
        else:
            self.display_topics(topics)

    def on_fallback_choice_change(self, change):
        clear_output()
        if change['new'] == 'Yes':
            fallback_topics = self.topic_manager.fetch_topics_from_file()
            if fallback_topics:
                self.display_topics(fallback_topics)
            else:
                print("Failed to load fallback topics.")
        else:
            print("Exiting the program.")

    def display_topics(self, topics):
        for topic in topics:
            self.assessment_manager.display_topic(topic)
            self.assessment_manager.conduct_assessment(topic)

if __name__ == "__main__":
    MainApp()
